# Program Description for Structure Descriptor Simulation (Module 2)

## 1. Module Overview
This module processes a cluster file in `.xyz` format containing `n` 3D structures (where `n` is the number of samples).

## 2. Functionalities
It integrates six commonly used neighbor atom calculation methods to compute coordination numbers (CN) and bond lengths (CR).

## 3. Implemented Methods
The following methods are implemented for calculating the structural descriptors:

- **EconNN**: Effective Coordination Number
- **VoronoiNN**: Voronoi Coordination Method
- **CrystalNN**: Crystal Nearest Neighbor Method
- **MinimumDistanceNN**: Minimum Distance Algorithm
- **JmolNN**: Jmol Algorithm
- **BrunnerNN**: Brunner Method

## 4. Additional Features
Additionally, the module provides an option to calculate the Radial Distribution Function (RDF) for each structure.

## 5. Output
- The calculated spectra are saved in a file named `datasets` in the current directory.
- Structure descriptors calculated by different methods are saved in separate files, named after the method used. For example, the coordination number calculated by the EconNN method is saved as `cn_EconNN` in the `cn` folder.

## 6. Progress Monitoring
This program includes a progress bar during the calculation process to monitor the progress of sample processing in real-time.

## 7. Parameters
- **`num_processes`**: Specifies the number of processes used for multi-process parallel computation.
- **`n`**: Defines the total number of samples to be processed.


#  Import libraries

In [1]:
import os
import pandas as pd
import multiprocessing
from multiprocessing import Lock, Pool, Manager
from tqdm import tqdm
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
#import pymatgen.analysis.local_env as getNN
from pymatgen.analysis.local_env import EconNN, VoronoiNN, CrystalNN, MinimumDistanceNN, JmolNN, BrunnerNN_relative
from ase.io import read, write
from os.path import join, exists, basename
from vasppy.rdf import RadialDistributionFunction
import glob
import re
import sys 
import pkg_resources

##  Version Information

In [2]:
def get_python_version():
    return sys.version
def get_package_version(package_name):
    try:
        module = __import__(package_name)
        version = getattr(module, '__version__', None)
        if version:
            return version
        else:
            return pkg_resources.get_distribution(package_name).version
    except (ImportError, AttributeError, pkg_resources.DistributionNotFound):
        return "Version info not found"

packages = ['pandas', 'tqdm', 'pymatgen', 'ase', 'vasppy', 're']
for package in packages:
    print(f"{package}: {get_package_version(package)}")
print(f"Python: {get_python_version()}")

pandas: 2.0.3
tqdm: 4.62.3
pymatgen: 2023.8.10
ase: 3.22.0
vasppy: 0.7.1.0
re: 2.2.1
Python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]


#  Parameter settings

## Parameters:

- **`num_processes`**: 
  - Used to manually set the number of processes for parallel computation.
  - Increasing this value allows the program to process more samples concurrently, potentially speeding up the computation for large datasets.

- **`n`**:
  - Defines the number of samples to be processed.
  - If you're unsure about the value, you can set this to a larger number to ensure that enough samples are processed. The program will handle any remaining samples.

- **`methods`**:
  - Specifies the calculation methods to be used for structure analysis.
  - Multiple methods can be selected, allowing users to choose different techniques for calculating structural descriptors (e.g., coordination number, bond length).
  - Users can select more than one method for a more comprehensive analysis.


In [3]:
# Format of the input data. Options are 'vasp-xdatcar' or 'lammps-dump-text'
format = "lammps-dump-text"

# Name of the input data file containing atomic structures
data_input = "Cu-20220624.xyz"

# Boolean flag indicating whether to read a single file or multiple files
read_single_file = True

# Keyword used to filter or identify specific elements or atoms in the data
keyword = "Cu"

# Base directory for storing various types of data and output files
data_dir = "1010-datasets"

# Directory for storing coordination number (CN) data
cn_dir = join(data_dir, "cn")

# Directory for storing crystal structure (CR) data
cr_dir = join(data_dir, "cr")

# Directory for storing radial distribution function (RDF) data in CSV format
rdf_dir_csv = join(data_dir, "rdf")

# Directory for storing POSCAR files (a file format used in VASP for atomic structures)
poscar_dir = join(data_dir, "poscar")

# Number of processes to use for parallel computation
num_processes = 5

# Parameter specifying the number of samples or iterations, used in computations 
n = 10

# List of methods for determining nearest neighbors or computing properties related to atomic structures
methods = [
    'BrunnerNN_relative',  # Relative nearest neighbor method
    'VoronoiNN',           # Voronoi tessellation-based nearest neighbor method
    'JmolNN',              # Nearest neighbor method from Jmol
    'MinimumDistanceNN',   # Simple nearest neighbor based on minimum distance
    'CrystalNN',           # Crystal-based nearest neighbor determination
    'EconNN'               # Efficient nearest neighbor determination
]

# Create the necessary directories if they do not already exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(cn_dir, exist_ok=True)
os.makedirs(cr_dir, exist_ok=True)
os.makedirs(rdf_dir_csv, exist_ok=True)
os.makedirs(poscar_dir, exist_ok=True)


In [4]:
def extract_number(filename):
    match = re.findall(r'\d+', basename(filename))
    numbers = [int(num) for num in match]
    return max(numbers) if numbers else float('inf')
if read_single_file:
    traj = read(data_input, format=format, index=':')
    total_samples = len(traj)
else:
    file_paths = sorted(glob.glob(join(data_input, '*.xyz')), key=extract_number)
    print("Sorted file paths:")
    for file_path in file_paths:
         print(file_path)
    trajectories = [read(file_path, format=format) for file_path in file_paths]
    total_samples = len(trajectories)
cpu_count = os.cpu_count()
print(f"CPU_numbers: {cpu_count}")
print(f"Sample_numbers: {total_samples}")
num_samples_to_process = min(n, total_samples)
num_processes=min(num_processes, cpu_count)
print(f"Structure calculation methods: {methods}")
print(f"Number of samples to process: {num_samples_to_process}")
print(f"Number of processes to use for parallel computing: {num_processes}")

CPU_numbers: 64
Sample_numbers: 5001
Structure calculation methods: ['BrunnerNN_relative', 'VoronoiNN', 'JmolNN', 'MinimumDistanceNN', 'CrystalNN', 'EconNN']
Number of samples to process: 10
Number of processes to use for parallel computing: 5


# Function settings

In [5]:
def compute_coordination(frame, absorbing_atom, structure, k):
    """Calculate the coordination number and average bond length"""
    label = {'index': k}
    for method_class in [BrunnerNN_relative, VoronoiNN, JmolNN, MinimumDistanceNN, CrystalNN, EconNN]:
        method = method_class()
        try:
            cn = method.get_cn(structure=structure, n=absorbing_atom)
            neighbors = method.get_nn_info(structure=structure, n=absorbing_atom)
            avg_bond_length = sum(neighbor['site'].distance(structure[absorbing_atom]) for neighbor in neighbors) / cn if cn > 0 else 0
            method_name = method.__class__.__name__
            label[f'CN_{method_name}'] = cn
            label[f'CR_{method_name}'] = avg_bond_length
        except Exception as e:
            print(f"Error with {method.__class__.__name__}: {e}")
            label[f'CN_{method_name}'] = None
            label[f'CR_{method_name}'] = None
    return label

def process_sample(args):
    """Processing a single sample"""
    j, frame, cn_dir, cr_dir, methods, lock = args
    try:
        frame.set_cell((40, 40, 40))
        structure = AseAtomsAdaptor.get_structure(frame)
        absorbing_atom = 0  # Assume the first atom is the absorbing atom
        results = compute_coordination(frame, absorbing_atom, structure, j)
        for method in methods:
            save_data({'index': results['index'], f'CN_{method}': results[f'CN_{method}']},
                      join(cn_dir, f"cn_{method}.csv"), lock)
            save_data({'index': results['index'], f'CR_{method}': results[f'CR_{method}']},
                      join(cr_dir, f"cr_{method}.csv"), lock)
    except Exception as e:
        print(f"Error processing sample {j}: {e}")

def save_data(label, save_path, lock):
    """Universal function to save data"""
    lock.acquire()
    try:
        if not exists(save_path):
            data = pd.DataFrame()
        else:
            data = pd.read_csv(save_path)
        print(f"Saving data to {save_path}: {label}")  # Debugging info
        data = pd.concat([data, pd.DataFrame([label])], ignore_index=True)
        data.to_csv(save_path, index=False)
    finally:
        lock.release()

def sort_and_save_csv(file_path):
    """Sort by index and save to CSV file"""
    if os.path.exists(file_path):
        data = pd.read_csv(file_path)
        data = data.sort_values(by="index")
        data.to_csv(file_path, index=False)
        print(f"Sorted and saved: {file_path}")
    else:
        print(f"File does not exist: {file_path}")

def compute_rdf(sample, index, rdf_dir_csv):
    """Calculate the radial distribution function and save it to a CSV file"""
    tmp_poscar = join(poscar_dir, f'POSCAR_{index}')
    write(tmp_poscar, sample, format='vasp', vasp5=True)
    
    clu = Structure.from_file(tmp_poscar)
    indices_1 = [i for i, site in enumerate(clu) if site.species_string == keyword and i != 0]
    
    rdf_cucu = RadialDistributionFunction(structures=[clu], indices_i=[0], indices_j=indices_1,
                                          r_min=1.0, r_max=8.0, nbins=100)
    
    rdf_data = pd.DataFrame({'r': rdf_cucu.r, 'g(r)': rdf_cucu.smeared_rdf()})
    rdf_data = rdf_data.round(3)
    csv_path = join(rdf_dir_csv, f'{index}.csv')
    rdf_data.to_csv(csv_path, index=False)
    print(f'RDF data has been saved to {csv_path}')
def sort_and_save_csv(file_path):
    if os.path.exists(file_path):
        data = pd.read_csv(file_path)
        data = data.sort_values(by="index")  # Sort by index
        data.to_csv(file_path, index=False)
        print(f"Sorted and saved: {file_path}")
    else:
        print(f"File does not exist: {file_path}")


# Main program

In [6]:
def main():
    with Manager() as manager:
        lock = manager.Lock()  # Create a lock using the Manager
        pool = Pool(processes=num_processes)

        if read_single_file:
            tasks = [(j, traj[j], cn_dir, cr_dir, methods, lock) for j in range(min(n, total_samples))]
        else:
            tasks = [(j, trajectories[j], cn_dir, cr_dir, methods, lock) for j in range(min(n, total_samples))]

        with tqdm(total=len(tasks), desc='Computing Samples') as pbar:
            for _ in pool.imap_unordered(process_sample, tasks):
                pbar.update(1)

        pool.close()
        pool.join()

        samples_to_process = traj[:n] if read_single_file else trajectories[:n]
        with Pool(processes=num_processes) as pool:
            results = pool.starmap(compute_rdf, [(samples_to_process[i], i, rdf_dir_csv) for i in range(len(samples_to_process))])

if __name__ == "__main__":
    main()

Computing Samples:   0%|                                 | 0/10 [00:00<?, ?it/s]/home/dell-hd/xraylarch/envs/hanxue2/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4141: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/dell-hd/xraylarch/envs/hanxue2/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3935: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/home/dell-hd/xraylarch/envs/hanxue2/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4141: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/dell-hd/xraylarch/envs/hanxue2/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3935: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic r

Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 4, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-datasets/cr/cr_BrunnerNN_relative.csv: {'index': 4, 'CR_BrunnerNN_relative': 2.4950970138707143}
Saving data to 1010-datasets/cn/cn_VoronoiNN.csv: {'index': 4, 'CN_VoronoiNN': 15}
Saving data to 1010-datasets/cr/cr_VoronoiNN.csv: {'index': 4, 'CR_VoronoiNN': 7.391947459363609}
Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 1, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-datasets/cn/cn_JmolNN.csv: {'index': 4, 'CN_JmolNN': 5}
Saving data to 1010-datasets/cr/cr_BrunnerNN_relative.csv: {'index': 1, 'CR_BrunnerNN_relative': 2.45508693148209}
Saving data to 1010-datasets/cr/cr_JmolNN.csv: {'index': 4, 'CR_JmolNN': 2.88691089571652}
Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 3, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 0, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-

Computing Samples:  10%|██▌                      | 1/10 [00:00<00:08,  1.12it/s]

Saving data to 1010-datasets/cn/cn_JmolNN.csv: {'index': 2, 'CN_JmolNN': 4}
Saving data to 1010-datasets/cn/cn_MinimumDistanceNN.csv: {'index': 3, 'CN_MinimumDistanceNN': 3}
Saving data to 1010-datasets/cn/cn_MinimumDistanceNN.csv: {'index': 0, 'CN_MinimumDistanceNN': 3}
Saving data to 1010-datasets/cn/cn_CrystalNN.csv: {'index': 1, 'CN_CrystalNN': 3}
Saving data to 1010-datasets/cr/cr_JmolNN.csv: {'index': 2, 'CR_JmolNN': 2.8083557980440395}
Saving data to 1010-datasets/cr/cr_MinimumDistanceNN.csv: {'index': 3, 'CR_MinimumDistanceNN': 2.6558849716556403}
Saving data to 1010-datasets/cr/cr_MinimumDistanceNN.csv: {'index': 0, 'CR_MinimumDistanceNN': 2.5526554800834367}
Saving data to 1010-datasets/cr/cr_CrystalNN.csv: {'index': 1, 'CR_CrystalNN': 2.45508693148209}
Saving data to 1010-datasets/cn/cn_MinimumDistanceNN.csv: {'index': 2, 'CN_MinimumDistanceNN': 3}
Saving data to 1010-datasets/cn/cn_CrystalNN.csv: {'index': 3, 'CN_CrystalNN': 3}
Saving data to 1010-datasets/cn/cn_CrystalNN.c

Computing Samples:  20%|█████                    | 2/10 [00:01<00:03,  2.31it/s]

...

Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 6, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-datasets/cn/cn_BrunnerNN_relative.csv: {'index': 8, 'CN_BrunnerNN_relative': 3}
Saving data to 1010-datasets/cr/cr_BrunnerNN_relative.csv: {'index': 6, 'CR_BrunnerNN_relative': 2.6529163714478172}
Saving data to 1010-datasets/cr/cr_BrunnerNN_relative.csv: {'index': 8, 'CR_BrunnerNN_relative': 2.637153941680596}
Saving data to 1010-datasets/cn/cn_VoronoiNN.csv: {'index': 6, 'CN_VoronoiNN': 16}
Saving data to 1010-datasets/cn/cn_VoronoiNN.csv: {'index': 8, 'CN_VoronoiNN': 15}
Saving data to 1010-datasets/cr/cr_VoronoiNN.csv: {'index': 6, 'CR_VoronoiNN': 7.673695311436043}
Saving data to 1010-datasets/cr/cr_VoronoiNN.csv: {'index': 8, 'CR_VoronoiNN': 7.536258635850855}
Saving data to 1010-datasets/cn/cn_JmolNN.csv: {'index': 6, 'CN_JmolNN': 3}
Saving data to 1010-datasets/cn/cn_JmolNN.csv: {'index': 8, 'CN_JmolNN': 3}
Saving data to 1010-datasets/cr/cr_JmolNN.csv: {'inde

Computing Samples:  70%|█████████████████▌       | 7/10 [00:01<00:00,  4.74it/s]

Saving data to 1010-datasets/cr/cr_EconNN.csv: {'index': 8, 'CR_EconNN': 2.637153941680596}
Saving data to 1010-datasets/cn/cn_MinimumDistanceNN.csv: {'index': 9, 'CN_MinimumDistanceNN': 3}
Saving data to 1010-datasets/cr/cr_JmolNN.csv: {'index': 7, 'CR_JmolNN': 2.7511119943102313}
Saving data to 1010-datasets/cr/cr_MinimumDistanceNN.csv: {'index': 9, 'CR_MinimumDistanceNN': 2.5660053838257535}
Saving data to 1010-datasets/cn/cn_MinimumDistanceNN.csv: {'index': 7, 'CN_MinimumDistanceNN': 3}
Saving data to 1010-datasets/cn/cn_CrystalNN.csv: {'index': 9, 'CN_CrystalNN': 3}
Saving data to 1010-datasets/cr/cr_MinimumDistanceNN.csv: {'index': 7, 'CR_MinimumDistanceNN': 2.5461632559234997}
Saving data to 1010-datasets/cr/cr_CrystalNN.csv: {'index': 9, 'CR_CrystalNN': 2.5660053838257535}
Saving data to 1010-datasets/cn/cn_CrystalNN.csv: {'index': 7, 'CN_CrystalNN': 3}
Saving data to 1010-datasets/cn/cn_EconNN.csv: {'index': 9, 'CN_EconNN': 3}
Saving data to 1010-datasets/cr/cr_CrystalNN.csv: 

Computing Samples: 100%|████████████████████████| 10/10 [00:01<00:00,  5.31it/s]


RDF data has been saved to 1010-datasets/rdf/1.csvRDF data has been saved to 1010-datasets/rdf/2.csv

RDF data has been saved to 1010-datasets/rdf/0.csvRDF data has been saved to 1010-datasets/rdf/4.csv

RDF data has been saved to 1010-datasets/rdf/3.csv
RDF data has been saved to 1010-datasets/rdf/5.csv
RDF data has been saved to 1010-datasets/rdf/8.csv
RDF data has been saved to 1010-datasets/rdf/6.csvRDF data has been saved to 1010-datasets/rdf/9.csv

RDF data has been saved to 1010-datasets/rdf/7.csv


## Sorting Output Files:

- After the calculations are completed, the resulting CSV files will be automatically sorted based on the sample index.
- This ensures that the output files are organized in a systematic order, corresponding to the sample index, making it easier to reference and analyze the results.
- Sorting is applied to all the output files, including the calculated spectra, coordination numbers, and other structural descriptors.


In [7]:
methods = ['BrunnerNN_relative', 'VoronoiNN', 'JmolNN', 'MinimumDistanceNN', 'CrystalNN', 'EconNN']
for method in methods:
    cn_file_path = os.path.join(cn_dir, f"cn_{method}.csv")
    sort_and_save_csv(cn_file_path)
for method in methods:
    cr_file_path = os.path.join(cr_dir, f"cr_{method}.csv")
    sort_and_save_csv(cr_file_path)

Sorted and saved: 1010-datasets/cn/cn_BrunnerNN_relative.csv
Sorted and saved: 1010-datasets/cn/cn_VoronoiNN.csv
Sorted and saved: 1010-datasets/cn/cn_JmolNN.csv
Sorted and saved: 1010-datasets/cn/cn_MinimumDistanceNN.csv
Sorted and saved: 1010-datasets/cn/cn_CrystalNN.csv
Sorted and saved: 1010-datasets/cn/cn_EconNN.csv
Sorted and saved: 1010-datasets/cr/cr_BrunnerNN_relative.csv
Sorted and saved: 1010-datasets/cr/cr_VoronoiNN.csv
Sorted and saved: 1010-datasets/cr/cr_JmolNN.csv
Sorted and saved: 1010-datasets/cr/cr_MinimumDistanceNN.csv
Sorted and saved: 1010-datasets/cr/cr_CrystalNN.csv
Sorted and saved: 1010-datasets/cr/cr_EconNN.csv
